# [ATL16](https://nsidc.org/data/atl16/)

* [ATB](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/ICESat2_ATL16_ATL17_ATBD_r004.pdf)


## Downloading data using earthaccess

In [ ]:
import earthaccess

auth = earthaccess.login()
earthaccess.__version__

In [ ]:
temporal_range = ("2022-01", "2022-03")
max_files = 10

results = earthaccess.search_data(short_name="ATL16",
                                  temporal = temporal_range,
                                  count=max_files)
files = earthaccess.download(results, local_path="../data/ATL16")

## Loading data

In [ ]:
import xarray as xr
import pathlib
import h5py
from jupyterlab_h5web import H5Web

In [ ]:
files = []
for file in pathlib.Path('../data/ATL16').glob('*.h5'):
    files.append(str(file))
files

## Exploring ATL16 with xarray

* A) Opening a single granule
> xarray uses h5py or netcdf4 to open `.h5` and `.nc` files, it opens the file and based on the signature it selects the backend
> If the files are local this should be pretty straight-forward. 
* B) Opening multiple granules and concatenating them by time (start or end) 
> Because this dataset does not have a time dimensionwe need to explicit include it if we want to concatenate mutiple granules
> We can do this using a preprocessing function

In [ ]:
# Single granule, xarray opens hdf5 files using the netcdf4/h5py libraries.
ds = xr.open_dataset(files[0])
ds

In [ ]:
# uses multiple grids
ds.dims

### Opening multiple granules (building a data cube)

In [ ]:

def preprocess(ds):
    # expand the dimentions to include time so we can concatenate the granules using the time dimension
    ds = ds.expand_dims(time=ds.delta_time_end.values)
    return ds


stacked_ds = xr.open_mfdataset(files, preprocess=preprocess, concat_dim='time', combine="nested")
stacked_ds

## Validating 

In [ ]:
## TODO

## Plotting

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# matplotlib plot with xarray
variable = "combined_global_cloud_frac"

stacked_ds[variable].plot.imshow(size=6, col="time", col_wrap=3)


In [ ]:
%matplotlib widget
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

viewpoint = (-90, 45)
variable = "combined_global_cloud_frac"

fig, axs = plt.subplots( subplot_kw=dict(projection=ccrs.Orthographic(viewpoint[0], viewpoint[1])))

p = ds[variable].plot(ax=axs, transform=ccrs.PlateCarree())
p.axes.coastlines()
p.axes.gridlines()
p.axes.set_global()
plt.show()